### Import dependencies-

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Data loading- 
df = pd.read_csv("customer_shopping_behavior.csv")

In [3]:
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,kurta,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,kurta,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [4]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   str    
 3   Item Purchased          3900 non-null   str    
 4   Category                3900 non-null   str    
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   str    
 7   Size                    3900 non-null   str    
 8   Color                   3900 non-null   str    
 9   Season                  3900 non-null   str    
 10  Review Rating           3863 non-null   float64
 11  Subscription Status     3900 non-null   str    
 12  Shipping Type           3900 non-null   str    
 13  Discount Applied        3900 non-null   str    
 14  Promo Code Used         3900 non-null   str    
 15

In [5]:
df.describe()

,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,3900.000000,3900.000000,3900.000000,3863.000000,3900.000000
mean,1950.500000,44.068462,59.764359,3.750065,25.351538
std,1125.977353,15.207589,23.685392,0.716983,14.447125
min,1.000000,18.000000,20.000000,2.500000,1.000000
25%,975.750000,31.000000,39.000000,3.100000,13.000000
50%,1950.500000,44.000000,60.000000,3.800000,25.000000
75%,2925.250000,57.000000,81.000000,4.400000,38.000000
max,3900.000000,70.000000,100.000000,5.000000,50.000000


In [6]:
df.columns

Index(['Customer ID', 'Age', 'Gender', 'Item Purchased', 'Category',
       'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season',
       'Review Rating', 'Subscription Status', 'Shipping Type',
       'Discount Applied', 'Promo Code Used', 'Previous Purchases',
       'Payment Method', 'Frequency of Purchases'],
      dtype='str')

In [7]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')

In [8]:
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount_(usd)', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases'],
      dtype='str')

In [9]:
# Rename columns-
df = df.rename(columns={'purchase_amount_(usd)':'purchase_amount'})

In [10]:
# Create new column- Age group
labels = ['Young','Adult','Mid-Age','Senior']
df['age_group'] = pd.qcut(df['age'],q=4,labels=labels)

In [11]:
df[['age','age_group']].head()

,age,age_group
0,55,Mid-Age
1,19,Young
2,50,Mid-Age
3,21,Young
4,45,Mid-Age


In [12]:
df['frequency_of_purchases'].value_counts()

frequency_of_purchases
Quarterly      1147
Annually        572
Monthly         553
Bi-Weekly       547
Fortnightly     542
Weekly          539
Name: count, dtype: int64

In [13]:
# Create new column Purchase frequency days-

frequency_map = {
    'Quarterly':90,
    'Monthly':30,
    'Weekly':7,
    'Fortnightly':14,
    'Annually':365,
    'Bi-Weekly':14
}
df['frequency_purchase_days']= df['frequency_of_purchases'].map(frequency_map)

In [14]:
df[['frequency_of_purchases','frequency_purchase_days']].head()

,frequency_of_purchases,frequency_purchase_days
0,Fortnightly,14
1,Fortnightly,14
2,Weekly,7
3,Weekly,7
4,Annually,365


In [15]:
# Dropping column with same values-
df = df.drop('promo_code_used',axis=1)

In [16]:
# Move dataframe to sql server-
from sqlalchemy import create_engine
Server= r"HP-255G10\SQLEXPRESS"
Database="db_Customer"
Driver="ODBC+17+FOR+SQLSERVER"
Database_con=( "mssql+pyodbc://HP-255G10\\SQLEXPRESS/db_Customer"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
    "&TrustServerCertificate=yes")
engine=create_engine(Database_con)
conn=engine.connect()

table_name="customers"
df.to_sql(table_name,engine,if_exists="replace",index=False)
print("Data loaded")

Data loaded
